In [2]:
import numpy as np 
from numpy import random
import pandas as pd 
#import seaborn as sns
import scipy 
from scipy.stats import chi2
import math
import sys
from scipy.stats import beta
import matplotlib.pyplot as plt



### 1.  Mixture of Beta

In [14]:

n= 5; p = 10
q = 0           # Rang de M. q = 0 revient simuler U = Y = E Gaussien nxp, car M = 0                        
m = n-q ;N=   1000  #Monte carlo nb de simulation

alpha= p/2 + .1 #alpha > p\2
Beta=  .00001    #beta > 0

b=1
c=(alpha- p/2 ) / (alpha - p/2 + Beta)
K_st_st=beta.moment(2, alpha, Beta, loc=0, scale=1)


a_Quad_loss = 1 / (K_st_st*( m + p + 1))
t_K_Quad_loss =  ( 2*(p-m-1)*( (p+m+1)*((c/b)**2) - p - 2 ) )/((p-m+1)*(p-m+3) )


##=============== 1/ Simuler le modele additif Y=M+E    =======================
#================ 1.1 \ Construction de la matrice des bruits E ===============

Loss_Sigma_a_Quad = 0
Loss_Sigma_K_Quad = 0 

E = np.zeros((n,p))
Sigma=np.zeros((p,p))

#------------Structure de Sigma-----------
Sigma=np.identity(p)
"-----------------------------------------"
#e =p*np.random.rand(p,p)
#Sigma= np.dot(e.T,e)
"-----------------------------------------"
"-----------------------------------------"
#gamma=1.9
#sigma=np.zeros(p)
#for i in range(0,p):
#    sigma[p-1-i]=gamma**i
#Sigma=np.diag(sigma)
"-----------------------------------------"
#Sigma= np.zeros((p,p))
#rho=0.9

#for ii in range(0,p):
#    for jj in range(0,p):
#        Sigma[ii,jj] = rho**abs(ii-jj)
"-----------------------------------------"




for j in range(0,N): 
    R=np.sqrt(np.random.beta(alpha, Beta, p));
    RR=np.tile(R,(n, 1))
    X=np.random.multivariate_normal(np.zeros(p), Sigma, n)
    E= np.multiply(X,RR)
    

#================ 1.2\ Creer une matrice M de rang q ==========================
   
    M0 = np.random.rand(n,p)                    # Generer une matrice aleatoire faire une QR pour M0
    [u,val_propre_M0,v] = np.linalg.svd(M0)        # Faire une S.V.D
    M = np.dot(np.dot(u[:,0:q],np.diag(val_propre_M0)[0:q,0:q]),v[0:q,:])
    [Q,R]=np.linalg.qr(M)
    Q_trans_Y = np.dot(Q.T,M) + np.dot(Q.T,E)
    Z = Q_trans_Y[0:q,:]
    U = Q_trans_Y[q:n,:] 
    #Y = M + E 
    
#==================== 2\ Forme canonique du modele ============================ 
    
    S = np.dot(U.T,U) 
    S_plus = np.linalg.pinv(S)                   # The Moore-Penrose inverse
    trace_S_plus = np.matrix.trace(S_plus)       # Trace de S^+
    mu = 1/ trace_S_plus
    [R,V,D] = np.linalg.svd(U)                   # SVD pour U afin de construire S= H L_tilde H^T
    #val_propre_S = V*V.T
    L = np.diag(V*V )                          # Recuperer L
    #L_inv= np.linalg.inv(L)
    
    L_tilde = np.zeros((p,p))
    H1 = D.T[:,0:m]
    
    #=============== 3\ Construction des estimateurs et des risques associes=======
   
    Sigma_hat_a_Quad_loss  =   a_Quad_loss *S
    risk_Sigma_hat_a_Quad_loss = p*(p+1)*a_Quad_loss                  #Sigma^_a
    
    #=============== Estimateur de Konno ===============
    

    Phi_K_Quad = L + mu * t_K_Quad_loss * np.identity(m)
    
    Sigma_hat_K =  a_Quad_loss  * np.dot( np.dot(H1,Phi_K_Quad), H1.T ) # Estimateur de Konno
 
    
   
    Loss_Sigma_K_Quad= Loss_Sigma_K_Quad +np.matrix.trace( ( ( (np.linalg.inv(Sigma))@ (Sigma_hat_K ) )-np.identity(p) ) @  ( ( (np.linalg.inv(Sigma))@ (Sigma_hat_K ) )-np.identity(p) ))
    Risk_Sigma_K_Quad=(1/N)*(Loss_Sigma_K_Quad)
    
    Loss_Sigma_a_Quad = Loss_Sigma_a_Quad + np.matrix.trace( ( ( (np.linalg.inv(Sigma))@ (Sigma_hat_a_Quad_loss) )-np.identity(p) ) @  ( ( (np.linalg.inv(Sigma))@ (Sigma_hat_a_Quad_loss ) )-np.identity(p) ))
    Risk_Sigma_a_Quad=(1/N)*(Loss_Sigma_a_Quad)
   

Prial_K_a_Quad = 100*( (Risk_Sigma_a_Quad - Risk_Sigma_K_Quad   ) / Risk_Sigma_a_Quad)

print('Cout Q:','Risk aS =',Risk_Sigma_a_Quad,"||" ,'risk Konno =', Risk_Sigma_K_Quad,"||",'Prial  = ',Prial_K_a_Quad)



Cout Q: Risk aS = 6.872320927406447 || risk Konno = 6.7418630611107995 || Prial  =  1.8983087034743753


### 2. Multivariate Gaussian

In [25]:

n= 5;
p = 10
q =   0      # Rang de M. q = 0 revient simuler U = Y = E Gaussien nxp, car M = 0                        
m = n-q ;N=   1000  #Monte carlo nb de simulation

b=1
c=1
K_st_st=1

a_Quad_loss = 1 / (K_st_st*( m + p + 1))
t_K_Quad_loss =  ( 2*(p-m-1)*( (p+m+1)*((c/b)**2) - p - 2 ) )/((p-m+1)*(p-m+3) )

#t_K_Quad_loss =  ( (p+m+1)*(m-1) )/((p-m+1)*(p-m+3) )


##=============== 1/ Simuler le modele additif Y=M+E    =======================
#================ 1.1 \ Construction de la matrice des bruits E ===============

Loss_Sigma_a_Quad = 0
Loss_Sigma_K_Quad = 0 

E = np.zeros((n,p))
Sigma=np.zeros((p,p))

#------------Structure de Sigma-----------
"-------------Identity----------------------"
Sigma=np.identity(p)
"------------Random--------------------"
#e =p*np.random.rand(p,p)
#Sigma= np.dot(e.T,e)
"-----------Block----------------------"
#gamma=1.9
#sigma=np.zeros(p)
#for i in range(0,p):
#    sigma[p-1-i]=gamma**i
#Sigma=np.diag(sigma)
"----------AR-----------------------"
#Sigma= np.zeros((p,p))
#rho=0.9

#for ii in range(0,p):
#    for jj in range(0,p):
#        Sigma[ii,jj] = rho**abs(ii-jj)
"-----------------------------------------"



for j in range(0,N): 
    #R=np.sqrt(np.random.beta(alpha, Beta, p));
    #RR=np.tile(R,(n, 1))
    #X=np.random.multivariate_normal(np.zeros(p), Sigma, n)
    #E= np.multiply(X,RR)
    E=np.random.multivariate_normal(np.zeros(p), Sigma, n)

#================ 1.2\ Creer une matrice M de rang q ==========================
   
    M0 = np.random.rand(n,p)                    # Generer une matrice aleatoire faire une QR pour M0
    [u,val_propre_M0,v] = np.linalg.svd(M0)        # Faire une S.V.D
    M = np.dot(np.dot(u[:,0:q],np.diag(val_propre_M0)[0:q,0:q]),v[0:q,:])
    [Q,R]=np.linalg.qr(M)
    Q_trans_Y = np.dot(Q.T,M) + np.dot(Q.T,E)
    Z = Q_trans_Y[0:q,:]
    U = Q_trans_Y[q:n,:] 
    #Y = M + E 
    
#==================== 2\ Forme canonique du modele ============================ 
    
    S = np.dot(U.T,U) 
    S_plus = np.linalg.pinv(S)                   # The Moore-Penrose inverse
    trace_S_plus = np.matrix.trace(S_plus)       # Trace de S^+
    mu = 1/ trace_S_plus
    [R,V,D] = np.linalg.svd(U)                   # SVD pour U afin de construire S= H L_tilde H^T
    #val_propre_S = V*V.T
    L = np.diag(V*V )                          # Recuperer L
    #L_inv= np.linalg.inv(L)
    
    L_tilde = np.zeros((p,p))
    H1 = D.T[:,0:m]
    
    #=============== 3\ Construction des estimateurs et des risques associes=======
   
    Sigma_hat_a_Quad_loss  =   a_Quad_loss *S
    risk_Sigma_hat_a_Quad_loss = p*(p+1)*a_Quad_loss                  #Sigma^_a
    
    #=============== Estimateur de Konno ===============
    

    Phi_K_Quad = L + mu * t_K_Quad_loss * np.identity(m)
    
    Sigma_hat_K =  a_Quad_loss  * np.dot( np.dot(H1,Phi_K_Quad), H1.T ) # Estimateur de Konno
 
    
   
    Loss_Sigma_K_Quad= Loss_Sigma_K_Quad +np.matrix.trace( ( ( (np.linalg.inv(Sigma))@ (Sigma_hat_K ) )-np.identity(p) ) @  ( ( (np.linalg.inv(Sigma))@ (Sigma_hat_K ) )-np.identity(p) ))
    Risk_Sigma_K_Quad=(1/N)*(Loss_Sigma_K_Quad)
    
    Loss_Sigma_a_Quad = Loss_Sigma_a_Quad + np.matrix.trace( ( ( (np.linalg.inv(Sigma))@ (Sigma_hat_a_Quad_loss) )-np.identity(p) ) @  ( ( (np.linalg.inv(Sigma))@ (Sigma_hat_a_Quad_loss ) )-np.identity(p) ))
    Risk_Sigma_a_Quad=(1/N)*(Loss_Sigma_a_Quad)
   

Prial_K_a_Quad = 100*( (Risk_Sigma_a_Quad - Risk_Sigma_K_Quad   ) / Risk_Sigma_a_Quad)

print('Cout Q:','Risk aS =',Risk_Sigma_a_Quad,"||" ,'risk Konno =', Risk_Sigma_K_Quad,"||",'Prial  = ',Prial_K_a_Quad)




Cout Q: Risk aS = 6.868697304558559 || risk Konno = 6.738398284004904 || Prial  =  1.8969975641113048


### 3. Comparaison of Prials for different structure of $\Sigma$
######  Les premieres lignes sont les Prials pour le melange de Beta et les deuxiemes pour la loi normal 



| n  | $(\Sigma)_{i,i}=1$ | $\Sigma_{i,j}=AR(.9)$    | $(\Sigma)_{i,i}=1.9^i$ |
|----|:------------------:|-------------------------:|------------------------|
| 10 |       0.1176       |          0.1169          |         0.0311         |
| 10 |       0.1231       |          0.1137          |         0.0264         |
                                                |                        |
| 15 |       0.2553       |          0.2551          |         0.0831         |
| 15 |       0.2684       |          0.2538          |         0.0779         |
                                                 |                        |
| 20 |       0.4294       |          0.4288          |         0.1323         |
| 20 |       0.4437       |          0.4325          |         0.1276         |
                                                 |                        |
| 25 |       0.6219       |          0.6317          |         0.2131         |
| 25 |       0.6498       |          0.6343          |         0.2021         |
                                                  |                        |
| 29 |       0.7878       |          0.8098          |         0.2528         |
| 29 |       0.8205       |          0.8058          |         0.2331         |

*Parametrages:

- Pour le mélange Beta les parametres : $\alpha=p/2 + 0.1$ et $\beta=(0.001)$
- p = 60 
- Pour le mélange Beta, le rapport c/b est proche de 1  (0.999)

*Remarques: 

- Les Prials augmente lorsque la taille de l'echantillon augmente.
- La differences des Prials entre la loi normal et le melange beta n'est pas significative
- Les meilleurs Prials sont obtenus dans le cas Gaussian pour $\Sigma=I_p$ 

### 4. Comparaison of Prials for  $\Sigma=I_p$

| (n,p)    | $\Sigma=I_p$       |
|----------|:------------------:|
| (5,16)   |       0.8485       |
| (5,16)   |       0.8574       |
          |                    |
| (10,31)  |       0.6714       |
| (10,31)  |       0.6694       |
          |                    |
| (20,61)  |       0.4259       |
| (20,61)  |       0.4264       |
          |                    |
| (30,91)  |       0.2424       |
| (30,91)  |       0.3100       |
          |                    |
| (40,121) |       0.2430       |
| (40,121) |       0.2426       |


* Remarques :
- Plus la difference entre p et n et grand plus les PRIALS diminue

| (p=151 | $(\Sigma)_{i,i}$=1 |
|--------|:------------------:|
| 5      |       0.0021       |
| 5      |       0.0020       |
|        |                    |
| 10     |       0.0089       |
| 10     |       0.0092       |
|        |                    |
| 20     |       0.0367       |
| 20     |       0.0366       |
|        |                    |
| 30     |       0.0794       |
| 30     |       0.0795       |
|        |              |
| 40     |       0.1343       |
| 40     |       0.1344       |

In [2]:
np.identity(4)

AttributeError: module 'numpy' has no attribute 'identity'